In [1]:
import pandas as pd
from datasets import Dataset 

In [ ]:
# load data into datafram 

title_label = pd.read_csv('title_labels.txt')
title_label = pd.DataFrame(title_label, columns=['Title', 'Label'])
title_label_dataset = Dataset.from_pandas(title_label)

# Convert label column to integer
title_label_dataset = title_label_dataset.map(lambda x: {'Label': int(x['Label'])})


In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments

tokenizer = BertTokenizer.from_pretrained("yassinemaaroufi/finbert")

model = BertForSequenceClassification.from_pretrained("yassinemaaroufi/finbert", num_labels = 3)

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples['Title'], padding = 'max_length', truncation=True)

tokenized_dataset = title_label_dataset.map(tokenize_function, batched=True)

In [ ]:
training_args = TrainingArguments(output_dir='./results', evaluation_strategy='epoch', learning_rate=2e-5, 
                                  per_device_train_batch_size=8, per_device_eval_batch_size=8, 
                                  num_train_epochs=3, weight_decay=0.01)


In [ ]:
trainer = Trainer(model=model,
                  args=training_args
                  train_dataset=tokenized_dataset,
                  eval_dataset=tokenized_dataset)

trainer.train()

In [ ]:
model.save_pretrained('./fin_bert_3_labels')
tokenizer.save_pretrained('./fin_bert_3_labels')


In [ ]:
predictions = trainer.predict(tokenized_dataset)
print(predictions)